In [ ]:
import Control.Monad.ST
import Data.Vector.Mutable as MV (swap)
import qualified Data.Vector.Generic as MV (thaw, freeze, fromList, toList)
import System.Random.MWC

In [ ]:
bodyST g = uniformR (0, 10) g

In [ ]:
:info bodyST

In [ ]:
randomST :: IO Int
randomST = withSystemRandom . asGenST $ bodyST

In [ ]:
shuffleList :: (Show a, NFData a) => [a] -> [a]
shuffleList (a:[]) = [a]
shuffleList xs =  runST $ withSystemRandom . asGenST $ \g -> do
    v <- MV.thaw $ MV.fromList xs
    v' <- modify v g n
    r <- MV.toList <$> MV.freeze v'
    return r
        where
            n = length xs
            modify v g 0 = return v
            modify v g i = do
                let i' = i - 1
                j <- uniformR (0,n) g
                MV.swap v i' j
                modify v g i'